In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [5]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = 'data'

In [6]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [7]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [8]:
data.shape

(1987, 5)

In [9]:
data.head(3)

,content,keywords,summary,title,url
0,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","[школа, образовательные стандарты, литература, история, фгос]","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html
1,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...","[красота, законы]",О живительной пользе укорота при выборе между плохим и хорошим,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html
2,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...","[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [11]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [12]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [13]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [14]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [15]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [16]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [17]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                      [ольга, васильева, обещала, "нг", не, перегружать, школьников]
1                                           [у, красоты, собственные, закон, и, воля]
2                                                            [апокалиптический, бунт]
3                   [f1., предсказать, результаты, гран-при, испании, было, несложно]
4                                               [возвращение, в, небесное, отечество]
5                                       [практическая, медицина, с, большим, будущим]
6                     [бумажный, носитель., вселенная,, человек,, язык, человеческий]
7                                      [в, багдаде, неспокойно,, а, будет, еще, хуже]
8    [ученые, предсказывают, появление, интуиции, у, искусственных, нейронных, сетей]
9                                  [россияне, от, вступления, в, вто, не, пострадают]
Name: title, dtype: object

In [18]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                            [и, в, –, по, на, что, с, о, к, не]
1                 [и, в, не, что, –, на, красота, вы, если, как]
2                     [в, и, не, на, –, от, о, из, пепеляев, но]
3                        [в, не, и, на, что, но, это, как, с, к]
4                          [и, в, –, не, он, с, на, что, а, его]
5                       [и, в, на, –, с, что, не, это, как, уже]
6                           [–, и, в, не, на, из, по, что, с, а]
7                       [в, и, не, на, но, ли, ираке, что, -, а]
8                           [и, в, на, о, с, –, как, а, что, ai]
9    [в, на, по, и, что, кристалина, –, россии, георгиева, том,]
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [21]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [22]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [23]:
data['content_norm'] = data['content'].apply(normalize)

In [24]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.1


Ещу улучшения!

In [25]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0    [стандарт, источник, фгоса, ольга, васильев, результат, предмет, школа, письмо, произведение]
1                      [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руина, место, мечта]
2                    [пепеляев, юзеф, книга, якутия, восстание, год, война, место, леонид, сибирь]
3                    [гонка, команда, пилот, сезон, время, машина, место, круг, гран-при, испания]
4                    [есенин, поэт, клюев, год, жизнь, смерть, сергей, человек, борода, мариенгоф]
5       [медицина, кафедра, выпускник, уровень, работа, год, практика, ординатор, рудна, обучение]
6               [книга, человек, мозг, говор, анатомия, глава, звезда, вопрос, слово, азербайджан]
7       [ирак, война, американец, войско, структура, принцип, взрыв, центр, террорист, количество]
8    [нейросеть, клетка, интеллект, время, основа, изображение, задача, работа, система, внимание]
9      [россия, вступление, кристалина, георгиев, вто, банка, страна, переговоры, директор, слово]
Name: cont

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [26]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [27]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [28]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [29]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [30]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [31]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [32]:
keywords[:3]

[['ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'произведение',
  'письмо',
  'редакция',
  'содержание'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'руина',
  'предмет',
  'нелепость',
  'занавеска',
  'мечта'],
 ['пепеляев',
  'юзеф',
  'якутия',
  'восстание',
  'книга',
  'леонид юзеф',
  'сибирь',
  'леонид',
  '20 ха',
  'красных']]

In [33]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другой.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 0.5 баллов).

## Способ 1 (+)

Можно попробовать добавить к существительным прилагательные в именительном падеже с сохранением числа и рода.

In [124]:
def find_noun_adjf(text):   
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.inflect({'nomn'}).word for word in words if word.tag.POS == 'NOUN' or word.tag.POS == 'ADJF']

    return words

In [126]:
data['content_norm_with_adj'] = data['content'].apply(find_noun_adjf)

In [454]:
data['content_norm_with_adj_str'] = data['content_norm_with_adj'].apply(' '.join)

tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
tfidf.fit(data['content_norm_with_adj_str'])

id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_with_adj_str'])

keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.27
F1 -  0.18
Jaccard -  0.1


F1 выросла, идем дальше.

## Способ 2 (+)

Можно попробовать вытащить существительные из заголовков и добавить их к нашим последним ключевым словам.

In [464]:
data[['title', 'keywords']][:20]

,title,keywords
0,"Ольга Васильева обещала ""НГ"" не перегружать школьников","[школа, образовательные стандарты, литература, история, фгос]"
1,У красоты собственные закон и воля,"[красота, законы]"
2,Апокалиптический бунт,"[юзефович, гражданская война, пепеляев, якутия]"
3,F1. Предсказать результаты Гран-При Испании было несложно,"[формула1, автоспорт, гонки, испания, квят]"
4,Возвращение в небесное отечество,"[есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]"
5,Практическая медицина с большим будущим,"[медвузы, медицинское образование, рудн, николай стуров, интервью]"
6,"Бумажный носитель. Вселенная, человек, язык человеческий","[литература, книги, периодика, космос, небо, астрономия, анатомия, филология]"
7,"В Багдаде неспокойно, а будет еще хуже","[сша, ирак, война]"
8,Ученые предсказывают появление интуиции у искусственных нейронных сетей,"[искусственный интеллект, робот, компьютер, технологии]"
9,Россияне от вступления в ВТО не пострадают,"[вб, вто, переговоры, тарифы]"


In [457]:
def find_noun(text):   
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.inflect({'nomn'}).word for word in words if word.tag.POS == 'NOUN']

    return words

In [458]:
data['title_norm'] = data['title'].apply(find_noun)

In [692]:
keywords2 = [a + b for a, b in zip(keywords, data['title_norm'])]

In [693]:
evaluate(data['keywords'], keywords2)

Precision -  0.14
Recall -  0.32
F1 -  0.19
Jaccard -  0.11


Есть улучшение еще на одну сотую.

## Cпособ 3 (-)

Можно попробовать вытащить из саммари имена, аббревиатуры (типа ГЭС), слова, написанные латиницей (youtube), слова, не распознанные парсером, и географические объекты (Псков), потому что все это часто попадает в ключевые слова.

In [549]:
data[['summary', 'keywords']][:10]

,summary,keywords
20,Лондон намерен представить уже второй пакет ограничений против РФ,"[британия, антироссийские санкции, отравление, скрипаль, химоружие, дипломаты, высылка]"
21,Как снизить факторы риска заболевания артрозом,"[суставы, артроз, заболевания, причины, симптомы, лечение]"
22,,"[конкурс, красоты, технополис, вдв, шойгу, путин, научная, рота, минобороны, инновации, технологии, ученые]"
23,,"[вгтрк, добродеев, реорганизация, дума]"
24,,"[украина, оон, резолюция, правозащита, крым, севастополь]"
25,Общественный штаб проконтролировал законность и прозрачность процедуры голосования в столице,"[президентские выборы, москва, наблюдатели, волонтеры, инциденты]"
26,"Все те же лица, тот же мыслей ход...","[путин, политика, экономика, выборы, президентские выборы, элита, власть]"
27,Судьбу проектов строительства ГЭС в Монголии должны решать ученые,"[экология, байкал, гэс, монголия, юнеско, иркутская область, бурятия, общественные слушания]"
28,Самолеты и субмарины обер-прокурора,"[история, россия, константин победоносцев, реформы, александр ii, александр iii, николай ii, политика, религия, флот, армия, империя, славянофилы, западники, александр блок, достоевский]"
29,Киев обрадовался катарскому газу,"[украина, россия, газ, газпром, нафтогаз, катар, порошенко]"


In [609]:
def find_stuff(text):   
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    final = []
    
    for word in words:
        if word != None and 'Surn' in word.tag or 'Geox' in word.tag or 'Name' in word.tag:
            try:
                final.append(word.inflect({'nomn'}).word)
            except:
                pass
        elif 'Abbr' in word.tag or 'LATN' in word.tag or 'UNKN' in word.tag:
            try:
                final.append(word.word)
            except:
                pass

    return final

In [647]:
data['summary_stuff'] = data['summary'].apply(find_stuff)

In [649]:
data['summary_stuff'][:20]

0                            []
1                            []
2                      [якутия]
3                   [формулы-1]
4              [сергей, есенин]
5                            []
6                            []
7                            []
8                            []
9                            []
10                     [россия]
11                         [бг]
12                      [псков]
13                           []
14                    [youtube]
15    [сша, британия, евросоюз]
16                           []
17                           []
18          [берлинал, хоакины]
19                           []
Name: summary_stuff, dtype: object

In [700]:
keywords3 = [list(set(a) | set(b)) for a, b in zip(keywords2, data['summary_stuff'])]

In [696]:
evaluate(data['keywords'], keywords3)

Precision -  0.14
Recall -  0.33
F1 -  0.19
Jaccard -  0.11


Бэйзлайн, конечно, все еще побивается, но этот способ не дал улучшений :(

(хотя немного выросла полнота)

## Способ 4 (-)

Сочетания "имя + фамилия" очень часто встречаются в ключевых словах. Можно использовать Наташу, чтобы корректно вытаскивать их и добавлять к нашим ключевым словам.

In [665]:
from natasha import PersonExtractor


def find_names(text):
    extractor = PersonExtractor()
    matches = extractor(text)
    names = []

    for match in matches:
        try:
            name = match.fact.name.first + ' ' + match.fact.name.last
            names.append(name)
        except:
            pass
        
    top = Counter(names).most_common(2)
    top = [elem[0] for elem in top]
    
    return top

In [671]:
data['content_names'] = data['content'].apply(find_names)

In [710]:
data['content_names'][:20]

0                               [ольга васильева, сергей волков]
1                                                             []
2           [леонид юзефович, анатолий пепеляев, леонид малышев]
3              [себастьян феттель, нико росберг, хэмилтон льюис]
4      [августа миклашевская, лев толстой, сергей александрович]
5                                                             []
6                      [анна урысон, П лебедева, яков перельман]
7                                                             []
8                 [николай карамзин, иммануил кант, иван павлов]
9     [кристалина георгиева, владимир ткаченко, алексей гордеев]
10                   [елена окопная, алексей герман, тыквер тот]
11                                                            []
12                   [лев толстой, иван сусанин, ярослав мудрый]
13                                [абу бакар, джек строу, май в]
14          [николай соболев, саша спилберг, владимир мединский]
15       [владимир путин,

Сейчас можно отсеять имена, которые были вытащены раньше и добавить к ключевым словам как полное имя, так и отдельно фамилию.

In [775]:
keywords4 = [[] for i in range(len(keywords3))]

for i, elem in enumerate(keywords3):
    for word in elem:
        check = False
        
        for name in data['content_names'][i]:
            if word in name:
                check = True
                
        if check == False:
            keywords4[i].append(word)
    
    for name in data['content_names'][i]:
        keywords4[i].append(name)
        keywords4[i].append(name.split()[1])

In [776]:
evaluate(data['keywords'], keywords4)

Precision -  0.12
Recall -  0.36
F1 -  0.17
Jaccard -  0.1


Бэйзлайн все еще побивается, но F1 упала.

## Способ 5

В датасете даны url статей. Статьи по разным тематикам публикают в разных разделах. Можно попробовать вытащить названия разделов из ссылок и посмотреть, есть ли какая-то корреляция с ключевыми разделами.

In [781]:
def find_cats(string):
    try:
        cat = re.search('.*?http://www.ng.ru/(.*?)/', string).group(1)
    except:
        cat = ''
        
    return cat

In [782]:
categories = data['url'].apply(find_cats)

Counter(categories)

Counter({'': 58,
         'accent': 1,
         'antiquare': 2,
         'antrakt': 23,
         'antrakt_plus': 1,
         'armament': 5,
         'armies': 33,
         'autosport': 68,
         'before': 12,
         'bogoslovie': 4,
         'capacity': 5,
         'chess': 1,
         'cinematograph': 41,
         'cis': 122,
         'columnist': 22,
         'concepts': 5,
         'courier': 38,
         'culture': 42,
         'dipkurer': 34,
         'ecology': 10,
         'economics': 130,
         'ecoprom': 20,
         'editorial': 25,
         'education': 29,
         'energy': 33,
         'events': 19,
         'facts': 19,
         'faith': 12,
         'fakty': 23,
         'five': 10,
         'glavkniga': 11,
         'health': 30,
         'history': 3,
         'ideas': 34,
         'internet': 1,
         'kafedra': 10,
         'kartblansh': 36,
         'kids': 10,
         'koncep': 2,
         'lit': 22,
         'moscow': 23,
         'n2018': 2,
       

Какие-то категории, типа style или world, слишком общие. Но можно попробовать использовать несколько очевидных.

In [787]:
def find_themes(string):
    try:
        cat = re.search('.*?http://www.ng.ru/(.*?)/', string).group(1)
    except:
        cat = ''
    
    themes = []
        
    if cat == 'autosport':
        themes.append('автоспорт')
    elif cat == 'ecoprom' or cat == 'ecology' or cat == 'ng_ecology' or cat == 'ng_ekologiya':
        themes.append('экология')
    elif cat == 'sochi2014':
        themes.append('олимпиада')
    elif cat == 'poetry':
        themes.append('поэзия')
             
    return themes

In [788]:
data['themes'] = data['url'].apply(find_themes)

Прибавим темы к предыдущему лучшему сету ключевых слов (из способа 3).

In [789]:
keywords5 = [list(set(a) | set(b)) for a, b in zip(keywords3, data['themes'])]

In [790]:
evaluate(data['keywords'], keywords5)

Precision -  0.15
Recall -  0.34
F1 -  0.2
Jaccard -  0.11


Даже несколько тем дали прирост на одну сотую - это неплохо.

Можно не добавлять теги руками, а использовать на вытащенных категориях пакет transliterate или API переводчиков.